In [1]:
import os
import random
import shutil
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import tensorflow as tf
from tensorflow.keras.preprocessing import image
import numpy as np
import time

train_datagen = ImageDataGenerator(
    rescale=1.0 / 255.0,
    rotation_range=30,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    validation_split=0.2  # Split data into 80% training and 20% validation
)

train_generator = train_datagen.flow_from_directory(
    'eyedataset',
    target_size=(224, 224),  # Adjust to the input size expected by ResNet-150
    batch_size=32,
    class_mode='categorical',  # For multi-class classification
    subset='training'
)

val_generator = train_datagen.flow_from_directory(
    'eyedataset',
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    subset='validation'
)


Found 3376 images belonging to 4 classes.
Found 841 images belonging to 4 classes.


In [2]:
import tensorflow as tf
from tensorflow.keras.applications import ResNet152
from tensorflow.keras import layers, Model
from tensorflow.keras.optimizers import Adam

# Load pre-trained ResNet-152 model
base_model = ResNet152(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Add custom classification head
x = base_model.output
x = layers.GlobalAveragePooling2D()(x)
x = layers.Dense(1024, activation='relu')(x)
predictions = layers.Dense(len(train_generator.class_indices), activation='softmax')(x)

# Create the model
resnet_model = Model(inputs=base_model.input, outputs=predictions)

# Freeze base layers
for layer in base_model.layers:
    layer.trainable = False

# Compile the ResNet model
resnet_model.compile(optimizer=Adam(lr=0.001), loss='categorical_crossentropy', metrics=['accuracy'])

# Train the ResNet model 
num_epochs = 10
history_resnet = resnet_model.fit(
    train_generator,
    steps_per_epoch=len(train_generator),
    epochs=num_epochs,
    validation_data=val_generator,
    validation_steps=len(val_generator)
)

# Save the ResNet model
resnet_model.save('resnet152_eye_disease.h5')


Epoch 1/10
106/106 [==============================] - 958s 9s/step - loss: 1.4538 - accuracy: 0.2977 - val_loss: 1.3521 - val_accuracy: 0.3353
Epoch 2/10
106/106 [==============================] - 981s 9s/step - loss: 1.3612 - accuracy: 0.3282 - val_loss: 1.6627 - val_accuracy: 0.2545
Epoch 3/10
106/106 [==============================] - 932s 9s/step - loss: 1.3727 - accuracy: 0.3418 - val_loss: 1.3483 - val_accuracy: 0.2996
Epoch 4/10
106/106 [==============================] - 1788s 17s/step - loss: 1.3231 - accuracy: 0.3614 - val_loss: 1.3324 - val_accuracy: 0.3722
Epoch 5/10
106/106 [==============================] - 995s 9s/step - loss: 1.3166 - accuracy: 0.3697 - val_loss: 1.4321 - val_accuracy: 0.2949
Epoch 6/10
106/106 [==============================] - 982s 9s/step - loss: 1.3175 - accuracy: 0.3759 - val_loss: 1.3697 - val_accuracy: 0.3294
Epoch 7/10
106/106 [==============================] - 990s 9s/step - loss: 1.3178 - accuracy: 0.3711 - val_loss: 1.3363 - val_accuracy: 0.33

In [3]:


# Load the trained ResNet model
resnet_model = tf.keras.models.load_model('resnet152_eye_disease.h5')

# Define a representative dataset for quantization (optional but recommended)
def representative_data_gen():
    for i in range(100):
    
            yield [tf.constant(train_generator[0][0][i:i+1])]

    



In [9]:
from concurrent.futures import ThreadPoolExecutor, as_completed
import threading  

# Load the trained ResNet model
resnet_model = tf.keras.models.load_model('resnet152_eye_disease.h5')

# Define the validation data generator
val_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1.0 / 255)

# Create a validation data generator
val_generator = val_datagen.flow_from_directory(
    'eyedataset',  # Path to your validation data directory
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical'
)

# Initialize variables for accuracy and inference time
total_correct = 0
total_samples = 0
total_inference_time = 0

# Set the number of batches or iterations you want to run
num_batches_to_evaluate = 50  # Adjust as needed

# Perform inference on the validation dataset for a specific number of batches
for batch_idx, batch in enumerate(val_generator):
    if batch_idx >= num_batches_to_evaluate:
        break  # Exit the loop after the specified number of batches

    images, labels = batch
    start_time = time.time()
    predictions = resnet_model.predict(images)
    end_time = time.time()
    inference_time = end_time - start_time
    predicted_classes = np.argmax(predictions, axis=1)
    total_correct += np.sum(predicted_classes == np.argmax(labels, axis=1))
    total_samples += len(images)
    total_inference_time += inference_time

    # Print progress
    print(f'Processed batch {batch_idx + 1}/{num_batches_to_evaluate}')


# Calculate accuracy and average inference time
accuracy_resnet = total_correct / total_samples
average_inference_time_resnet = total_inference_time / total_samples

print(f'ResNet Model Accuracy: {accuracy_resnet}')
print(f'Average Inference Time (ResNet): {average_inference_time_resnet} seconds')

Found 4217 images belonging to 4 classes.
1/1 [==============================] - 9s 9s/step
Processed batch 1/50
1/1 [==============================] - 6s 6s/step
Processed batch 2/50
1/1 [==============================] - 5s 5s/step
Processed batch 3/50
1/1 [==============================] - 6s 6s/step
Processed batch 4/50
1/1 [==============================] - 6s 6s/step
Processed batch 5/50
1/1 [==============================] - 7s 7s/step
Processed batch 6/50
1/1 [==============================] - 6s 6s/step
Processed batch 7/50
1/1 [==============================] - 6s 6s/step
Processed batch 8/50
1/1 [==============================] - 6s 6s/step
Processed batch 9/50
1/1 [==============================] - 6s 6s/step
Processed batch 10/50
1/1 [==============================] - 6s 6s/step
Processed batch 11/50
1/1 [==============================] - 6s 6s/step
Processed batch 12/50
1/1 [==============================] - 6s 6s/step
Processed batch 13/50
1/1 [==========================

In [14]:
# Convert to TFLite with static quantization
converter = tf.lite.TFLiteConverter.from_keras_model(resnet_model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.representative_dataset = representative_data_gen
quantized_tflite_model = converter.convert()

# Save the quantized model
with open("quantized_resnet152_static.tflite", "wb") as f:
    f.write(quantized_tflite_model)


INFO:tensorflow:Assets written to: C:\Users\ASUS\AppData\Local\Temp\tmp7ta4iezw\assets


INFO:tensorflow:Assets written to: C:\Users\ASUS\AppData\Local\Temp\tmp7ta4iezw\assets
C:\Users\ASUS\anaconda3\lib\site-packages\tensorflow\lite\python\convert.py:789: UserWarning: Statistics for quantized inputs were expected, but not specified; continuing anyway.
  warnings.warn("Statistics for quantized inputs were expected, but not "


In [15]:
# Load the quantized TFLite model
interpreter = tf.lite.Interpreter(model_path='quantized_resnet152_static.tflite')
interpreter.allocate_tensors()

# Get input and output tensors
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()
batch_size=32
# Define the validation data generator
val_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1.0 / 255)

# Create a validation data generator
val_generator = val_datagen.flow_from_directory(
    'eyedataset',  # Path to your validation data directory
    target_size=(224, 224),
    batch_size=batch_size,  # Set the batch size
    class_mode='categorical'
)

# Initialize variables for accuracy and inference time
total_correct = 0
total_samples = 0
total_inference_time = 0

# Set the number of batches you want to evaluate (adjust as needed)
num_batches_to_evaluate = 50

# Perform inference on batches of images
for batch_index in range(num_batches_to_evaluate):
    images, labels = val_generator.next()  # Get the next batch of images and labels

    # Set the batch size to 1
    images = images[:1]

    # Prepare the input tensor
    interpreter.set_tensor(input_details[0]['index'], images)

    start_time = time.time()
    interpreter.invoke()
    output_data = interpreter.get_tensor(output_details[0]['index'])
    end_time = time.time()

    inference_time = end_time - start_time
    predicted_classes = np.argmax(output_data, axis=1)

    # Compare predicted classes with true labels
    correct_predictions = np.sum(predicted_classes == np.argmax(labels, axis=1))
    total_correct += correct_predictions
    total_samples += batch_size
    total_inference_time += inference_time

    # Print progress
    print(f'Processed batch {batch_index + 1}/{num_batches_to_evaluate}')

# Calculate accuracy and average inference time
accuracy_quantized = total_correct / total_samples
average_inference_time_quantized = total_inference_time / total_samples

print(f'Quantized Model Accuracy (Static): {accuracy_quantized}')
print(f'Average Inference Time (Quantized Static): {average_inference_time_quantized} seconds')

Found 4217 images belonging to 4 classes.
Processed batch 1/50
Processed batch 2/50
Processed batch 3/50
Processed batch 4/50
Processed batch 5/50
Processed batch 6/50
Processed batch 7/50
Processed batch 8/50
Processed batch 9/50
Processed batch 10/50
Processed batch 11/50
Processed batch 12/50
Processed batch 13/50
Processed batch 14/50
Processed batch 15/50
Processed batch 16/50
Processed batch 17/50
Processed batch 18/50
Processed batch 19/50
Processed batch 20/50
Processed batch 21/50
Processed batch 22/50
Processed batch 23/50
Processed batch 24/50
Processed batch 25/50
Processed batch 26/50
Processed batch 27/50
Processed batch 28/50
Processed batch 29/50
Processed batch 30/50
Processed batch 31/50
Processed batch 32/50
Processed batch 33/50
Processed batch 34/50
Processed batch 35/50
Processed batch 36/50
Processed batch 37/50
Processed batch 38/50
Processed batch 39/50
Processed batch 40/50
Processed batch 41/50
Processed batch 42/50
Processed batch 43/50
Processed batch 44/50